In [104]:
import numpy as np
import pandas as pd
import pyspark as ps
from langdetect import detect

In [2]:
schema = pd.read_table('twitter-swisscom/schema.txt', header=None, delim_whitespace=True, index_col=0, names=['name', 'type', 'specification', '??', 'format'] )

In [106]:
schema.head()

,name,type,specification,??,format
1,id,bigint(20),UNSIGNED,No,None
2,userId,bigint(20),UNSIGNED,No,None
3,createdAt,timestamp,No,0000-00-00,00:00:00
4,text,text,utf8_unicode_ci,No,None
5,longitude,float,Yes,NaN,NaN


In [143]:
long_data = pd.read_table('twitter-swisscom/sample.tsv', 
              sep='\t', engine='c', encoding='utf-8', nrows=9000, 
              header=None, names=schema.name, na_values=['\\N', 'nan'])

len(long_data['longitude'].dropna())

1322

In [502]:
table = pd.read_table('twitter-swisscom/twex.tsv', 
              sep='\t', engine='c', encoding='utf-8', 
              header=None, names=schema.name, na_values=['\\N', 'nan'], skiprows=21782062, nrows=1)
table['text'] = table['text'].astype(str)
table.text = table.text.str.replace('http\S+|www.\S+', '', case=False)#remove website
table.text = table.text.str.replace('@\S+|via', '', case=False)#remove @ and via
table.text = table.text.str.replace('\((.+?)\)', '', case=False)#remove content in (
table.text = table.text.str.replace('([^\s\w]|_)+','', case=False)#remove non alphanumeric (needed for language dec)
#table.text.dropna(inplace=True)

In [421]:
pd.set_option('display.max_colwidth', -1)

0     traumfinal                                  
1    Sponsorentreffen twittboat                   
2     12 oder 13 aber ist keine zuverlässige info 
3    Und wie RT  Proud ger wc2010                 
4    RT  RT  RT                                   
Name: text, dtype: object

remove all websites from string as this hinders detection.
then only check for tweets that still contain aphabetic characters, others are spam anyway

In [469]:
table.text[table.text.map(lambda x:any(c.isalpha() for c in x))].map(detect).value_counts()

Series([], Name: text, dtype: int64)

In [ ]:
table.text.where(lambda x: any(c.isalpha() for c in x))

In [503]:
table.text

0    Bye bye la France 
Name: text, dtype: object

In [6]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext('local', 'pyspark')
sqlContext = SQLContext(sc)

In [29]:
df = sqlContext.read.format('com.databricks.spark.csv')\
.option("delimiter", "\t").options(header='false')\
.load('twitter-swisscom/twex.tsv')

In [13]:
len(df.columns)

20

Question:
 - how do we deal with this in Spark uniquely.
 - we will have to prefilter, we can't load everything in frame anyway
 - so best do it in spark, but how? what code will enable us to do this, eg how do we deal with this in spark...as we have it distributed & need to read the tsv file...how will this work.
 - there should be a way to use the schema file to help..but how

verify longitude, latitude non nan
remove userLocation -> is useless
remove sourceUrl
remove userName/screenName
remove sourceName
remove placeLatitude
remove placeLongitude

where is the sentiment analysis?


In [ ]:
#spark.read.csv(
#    "twitter-swisscom/sample.tsv", header=False, mode="DROPMALFORMED", schema=schema)